In [317]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import pgeocode
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [318]:
# Read HouseTS.csv into area_df
area_df = pd.read_csv('../raw_data/HouseTS.csv')

# Read realtor-data.csv into house_df
house_df = pd.read_csv('../raw_data/realtor-data.csv')

In [319]:
# Get the latest twelve month of 'median_sale_price' per 'zipcode' from area_df
last_twelve_month_df = area_df[area_df['date'] == '2023-12-31'].copy()
last_twelve_month_df = last_twelve_month_df[['zipcode', 'median_sale_price']]

In [320]:
# Create list of unique zipcodes in area_df
unique_zipcodes_area_df = area_df['zipcode'].unique().tolist()

# Filter house_df by unique_zipcoes_area_df
house_df = house_df[house_df['zip_code'].isin(unique_zipcodes_area_df)]

In [321]:
# Assuming filtered_house_df and last_twelve_month_df are your DataFrames
# Merge filtered_house_df with last_twelve_month_df on zip_code
merged_df = house_df.merge(
    last_twelve_month_df[['zipcode', 'median_sale_price']],
    left_on='zip_code',
    right_on='zipcode',
    how='left'
)

# Drop the redundant zipcode column (from last_twelve_month_df)
merged_df = merged_df.drop(columns=['zipcode'], errors='ignore')

In [ ]:
def clean_data(df):
    # Drop columns 'street', 'city', 'state' and 'prev_sold_date'
    df = df.drop(columns=['street', 'city', 'state', 'prev_sold_date', 'brokered_by', 'status'])

    # Drop duplicates
    df = df.drop_duplicates()

    # Drop rows with NaN values from 'price'
    df = df.dropna(subset=['price'])

    # Create list where 'bed' & 'bath' & 'house_size' are NaN
    nan_values = df[
        (pd.isna(df['bed'])) &
        (pd.isna(df['bath'])) &
        (pd.isna(df['house_size']))
    ]

    # Filter out rows that are in nan_values because we assume they are land sales
    df = df[~df.index.isin(nan_values.index)]

    # Impute missing data
    df['bed'] = df['bed'].fillna(df['bed'].median())
    df['bath'] = df['bath'].fillna(df['bath'].median())
    df['house_size'] = df['house_size'].fillna(df['house_size'].median())
    df['acre_lot'] = df['acre_lot'].fillna(0)

    # Step 2: Calculate PPSF for each row
    # No NaNs or zeros in price or house_size
    df['ppsf'] = df['price'] / df['house_size']

    # Step 3: Calculate median PPSF per zip_code
    ppsf_median = df.groupby('zip_code')['ppsf'].median().reset_index(name='ppsf_zipcode')

    # Step 4: Merge median PPSF back to df
    df = df.merge(ppsf_median, on='zip_code', how='left')

    # Drop temporary ppsf column
    df = df.drop(columns=['ppsf'])

    # Calculate boundaries for 'price', 'acre_lot', 'house_size'
    lower_price = df['price'].quantile(0.03)
    upper_price = df['price'].quantile(0.97)
    upper_house_size = df['house_size'].quantile(0.99)
    lower_acre_lot = df['acre_lot'].quantile(0.01)
    upper_acre_lot = df['acre_lot'].quantile(0.99)
    lower_ppsf_zipcode = df['ppsf_zipcode'].quantile(0.03)
    upper_ppsf_zipcode = df['ppsf_zipcode'].quantile(0.97)



    # Apply boundaries to clean_house_df
    df = df[
        (df['price'] > lower_price) &
        (df['price'] < upper_price) &
        (df['bed'] < 14) &
        (df['bath'] < 12) &
        (df['house_size'] < upper_house_size) &
        (df['median_sale_price'] > 50000) &
        (df['acre_lot'] > lower_acre_lot) &
        (df['acre_lot'] < upper_acre_lot) &
        (df['ppsf_zipcode'] > lower_ppsf_zipcode) &
        (df['ppsf_zipcode'] < upper_ppsf_zipcode)
        ]

    return df

In [323]:
# Clean df
cleaned_house_df = clean_data(merged_df)

In [ ]:
def convert_zipcode(df):
    # Convert zip_code column to 5-digit string
    df['zip_code'] = df['zip_code'].astype(str).str.replace('\.0$', '', regex=True).str.zfill(5)

    # Get unique zip codes
    unique_zips = df['zip_code'].unique()

    # Initialize pgeocode for US
    nomi = pgeocode.Nominatim('us')

    # Function to get coordinates
    def get_coordinates(zip_code):
        try:
            result = nomi.query_postal_code(zip_code)
            if result.empty or pd.isna(result.latitude):
                return pd.Series([None, None])
            return pd.Series([result.latitude, result.longitude])
        except:
            return pd.Series([None, None])

    # Create DataFrame for unique zip codes
    zip_coords = pd.DataFrame(unique_zips, columns=['zip_code'])
    zip_coords[['latitude', 'longitude']] = zip_coords.apply(lambda row: get_coordinates(row['zip_code']), axis=1)

    # Map coordinates back to filtered_house_df
    coords_dict = zip_coords.set_index('zip_code')[['latitude', 'longitude']].to_dict('index')
    df['latitude'] = df['zip_code'].map(lambda x: coords_dict.get(x, {}).get('latitude'))
    df['longitude'] = df['zip_code'].map(lambda x: coords_dict.get(x, {}).get('longitude'))

    # Drop 'zip_code' column
    df = df.drop(columns=['zip_code'])

    return df

In [325]:
# Convert zipcodes to coordinates
cleaned_house_df = convert_zipcode(cleaned_house_df)

In [326]:
cleaned_house_df.describe()

,price,bed,bath,acre_lot,house_size,median_sale_price,ppsf_zipcode,latitude,longitude
count,5.574490e+05,557449.000000,557449.000000,557449.000000,557449.000000,5.574490e+05,557449.000000,557449.000000,557449.000000
mean,5.856484e+05,3.432621,2.626326,0.422593,2066.074724,5.416666e+05,285.692955,36.379982,-95.088487
std,3.884620e+05,1.076993,1.038813,0.921489,924.998689,2.911262e+05,148.443836,5.386400,17.223505
min,1.152000e+05,1.000000,1.000000,0.010000,100.000000,6.400000e+04,124.877451,25.284600,-123.633500
25%,3.299000e+05,3.000000,2.000000,0.120000,1438.000000,3.493225e+05,181.096145,32.949200,-115.241900
50%,4.750000e+05,3.000000,2.000000,0.180000,1810.000000,4.620000e+05,236.378205,35.167700,-93.236100
75%,7.000000e+05,4.000000,3.000000,0.290000,2484.000000,6.500000e+05,331.816756,40.441100,-80.272500
max,2.594818e+06,13.000000,11.000000,9.990000,6667.000000,6.150000e+06,991.830986,48.239500,-70.619400


In [332]:
# Create X and y
X = cleaned_house_df.drop(columns=['price'])
y = cleaned_house_df['price']

# Extract features
features = X.columns.tolist()

# Preprocess with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features)  # Scale all numeric features
    ]
)
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Apply preprocessor
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
# Train XGBoost model
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1)
model.fit(X_train_transformed, y_train)
# Evaluate
y_pred = model.predict(X_test_transformed)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"XGBoost RMSE: ${rmse:,.2f}")
# Feature importance
print("Feature importance:")
for name, importance in zip(features, model.feature_importances_):
    print(f"{name}: {importance:.4f}")
# Save predictions
test_results = X_test.copy()
test_results['actual_price'] = y_test
test_results['predicted_price'] = y_pred
test_results.to_csv('price_predictions_xgboost.csv', index=False)

XGBoost RMSE: $159,166.23
Feature importance:
bed: 0.0152
bath: 0.0864
acre_lot: 0.0152
house_size: 0.2322
median_sale_price: 0.4291
ppsf_zipcode: 0.2015
latitude: 0.0103
longitude: 0.0101


In [334]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

# Work on a copy to avoid SettingWithCopyWarning
df = cleaned_house_df.copy()
df = df.drop(columns=['median_sale_price'])

# Define features and target
target = 'price'
features = [col for col in df.columns if col != target]  # Exclude price
numeric_features = [col for col in features if col != 'zip_code']  # Exclude zip_code

# Verify columns
print("\nFeatures:", features)
print("Numeric features for scaling:", numeric_features)
if target not in df.columns:
    raise ValueError(f"'{target}' column not found. Available columns: {df.columns.tolist()}")

# Create X and y
X = df[features]
y = df[target]

# Preprocess with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ],
    remainder='passthrough'  # Keep zip_code unscaled
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1))
])

# Train model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print results
print("\nModel Results:")
print(f"XGBoost RMSE: ${rmse:,.2f}")
print(f"Mean Absolute Error (MAE): ${mae:,.2f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
feature_names = numeric_features + ['zip_code']
print("\nFeature Importance:")
for name, importance in zip(feature_names, pipeline.named_steps['regressor'].feature_importances_):
    print(f"{name}: {importance:.4f}")

# Sample of actual vs. predicted prices
results_df = pd.DataFrame({
    'Actual Price': y_test,
    'Predicted Price': y_pred,
    'Difference': y_test - y_pred
})
print("\nSample of Actual vs. Predicted Prices:")
print(results_df.head(10))

# Save predictions
test_results = X_test.copy()
test_results['actual_price'] = y_test
test_results['predicted_price'] = y_pred
test_results.to_csv('price_predictions_xgboost.csv', index=False)
print("\nPredictions saved to 'price_predictions_xgboost.csv'")


Features: ['bed', 'bath', 'acre_lot', 'house_size', 'ppsf_zipcode', 'latitude', 'longitude']
Numeric features for scaling: ['bed', 'bath', 'acre_lot', 'house_size', 'ppsf_zipcode', 'latitude', 'longitude']

Model Results:
XGBoost RMSE: $159,853.26
Mean Absolute Error (MAE): $96,441.10
R² Score: 0.8312

Feature Importance:
bed: 0.0167
bath: 0.1017
acre_lot: 0.0200
house_size: 0.3618
ppsf_zipcode: 0.4774
latitude: 0.0114
longitude: 0.0111

Sample of Actual vs. Predicted Prices:
        Actual Price  Predicted Price    Difference
439437      774900.0     6.965196e+05   78380.43750
734885      758000.0     8.205723e+05  -62572.31250
116227      493000.0     4.319082e+05   61091.75000
40597       199000.0     3.468239e+05 -147823.93750
308827      442279.0     4.052805e+05   36998.50000
417800      537500.0     5.040342e+05   33465.75000
679328     1949000.0     1.775860e+06  173140.12500
480650      279000.0     3.479096e+05  -68909.62500
693389     1300000.0     1.079865e+06  220135.3750